In [15]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yaml
import yfinance as yf
import quantstats as qs

In [2]:
keys = yaml.safe_load(open('../keys.yaml', 'r'))

In [3]:
df = yf.download("BTC-USD", start="2010-01-01", end="2024-01-01")

# Display the data
df.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [4]:
fast_ma = vbt.MA.run(df['Close'], 37)
slow_ma = vbt.MA.run(df['Close'], 49)
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)
portfolio = vbt.Portfolio.from_signals(df['Close'], entries, exits, freq = 'd', direction = 'longonly')
portfolio.total_return()

46.53629688699568

In [5]:
portfolio.sharpe_ratio()

1.0442862084273556

In [6]:
returns = portfolio.daily_returns()
returns

Date
2014-09-17    0.000000
2014-09-18    0.000000
2014-09-19    0.000000
2014-09-20    0.000000
2014-09-21    0.000000
                ...   
2023-12-27    0.021694
2023-12-28   -0.018760
2023-12-29   -0.012397
2023-12-30    0.001366
2023-12-31    0.002569
Length: 3393, dtype: float64

In [7]:
returns.mean()

0.0015436572846669603

In [8]:
returns.mean()/returns.std()

0.05466043835218986

In [9]:
# sharpe ratio
mean_return = returns.mean()*365
volatility_return = returns.std()*np.sqrt(365)
(mean_return - 0.0)/volatility_return

1.0442862084273377

In [10]:
portfolio.sharpe_ratio(risk_free = 0.05)

-32.78078036561705

# sharpe ratio

In [18]:
np.sqrt(365)*(returns.mean()/returns.std()) # should be 252 as number of trading days, not calendar days, also - benchmark rate is missing

1.0442862084273377

In [19]:
np.sqrt(252)*(returns.mean()/returns.std())

0.8677075586020293

In [16]:
qs.stats.sharpe(returns)

0.8677075586020293

# calmar ratio

In [14]:
portfolio.calmar_ratio()

0.7899150293734258

In [27]:
qs.stats.calmar(returns)

0.509658124498354

In [21]:
help(qs.stats.calmar)

Help on function calmar in module quantstats.stats:

calmar(returns, prepare_returns=True)
    Calculates the calmar ratio (CAGR% / MaxDD%)



In [26]:


# Example daily returns data (replace with your actual data)
daily_returns = returns

# Calculate the geometric mean return
geometric_mean_return = np.prod(1 + daily_returns) ** (1 / len(daily_returns)) - 1

# Calculate the compound annual growth rate (CAGR)
beginning_value = 10000  # Initial investment value
ending_value = beginning_value * (1 + daily_returns).prod()  # Final investment value
investment_period_years = len(daily_returns) / 365  # 252
cagr = (ending_value / beginning_value) ** (1 / investment_period_years) - 1

# Calculate the maximum drawdown
cumulative_returns = (1 + daily_returns).cumprod()
peak = cumulative_returns.cummax()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min()

# Calculate the annualized Calmar ratio using both methods
calmar_geometric = geometric_mean_return / abs(max_drawdown)
calmar_cagr = cagr / abs(max_drawdown)

print("Annualized Calmar Ratio (Using Geometric Mean Return):", calmar_geometric)
print("Annualized Calmar Ratio (Using CAGR):", calmar_cagr)


Annualized Calmar Ratio (Using Geometric Mean Return): 0.001746683173361901
Annualized Calmar Ratio (Using CAGR): 0.7899150293734258


# sortino ratio

In [29]:
portfolio.sortino_ratio()

1.5272044867665482

In [60]:
qs.stats.sortino(returns, annualize=True, periods = 365)

1.5272044867665513

In [36]:
help(qs.stats.sortino)

Help on function sortino in module quantstats.stats:

sortino(returns, rf=0, periods=252, annualize=True, smart=False)
    Calculates the sortino ratio of access returns
    
    If rf is non-zero, you must specify periods.
    In this case, rf is assumed to be expressed in yearly (annualized) terms
    
    Calculation is based on this paper by Red Rock Capital
    http://www.redrockcapital.com/Sortino__A__Sharper__Ratio_Red_Rock_Capital.pdf

